In [27]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D

In [28]:
def branin(x, y):
    a, b, c, d, e, f = 1.0, 5.1 / (4 * np.pi ** 2), 5 / np.pi, 6, 10, 1 / (8 * np.pi)
    return a * (y - b * x ** 2 + c * x - d) ** 2 + e * (1 - f) * np.cos(x) + e

def easom(x, y):
    return -np.cos(x) * np.cos(y) * np.exp(-((x - np.pi) ** 2 + (y - np.pi) ** 2))

def goldstein_price(x, y):
    term1 = (1 + ((x + y + 1) ** 2) * (19 - 14*x + 3*x**2 - 14*y + 6*x*y + 3*y**2))
    term2 = (30 + ((2*x - 3*y) ** 2) * (18 - 32*x + 12*x**2 + 48*y - 36*x*y + 27*y**2))
    return term1 * term2

def six_hump_camel(x, y):
    return (4 - 2.1 * x ** 2 + (x ** 4) / 3) * x ** 2 + x * y + (-4 + 4 * y ** 2) * y ** 2


In [29]:

# Генетичний алгоритм
def genetic_algorithm(func, bounds, pop_size=20, generations=100, mutation_rate=0.1, crossover_rate=0.7):
    population = np.random.uniform(bounds[:, 0], bounds[:, 1], (pop_size, 2))
    best_points = []  
    all_populations = []  
    
    for gen in range(generations):
        fitness = np.array([func(ind[0], ind[1]) for ind in population])
        best_idx = np.argmin(fitness)
        best_points.append(tuple(population[best_idx]))
        all_populations.append(population.copy())
        
        selected = population[np.argsort(fitness)[:pop_size//2]]
        offspring = []
        for _ in range(pop_size // 2):
            p1, p2 = selected[np.random.choice(len(selected), 2, replace=False)]
            if np.random.rand() < crossover_rate:
                alpha = np.random.rand()
                child = alpha * p1 + (1 - alpha) * p2
                offspring.append(child)
            else:
                offspring.append(p1)
                offspring.append(p2)
        
        offspring = np.array(offspring)
        mutations = np.random.uniform(-0.1, 0.1, offspring.shape) * (np.random.rand(*offspring.shape) < mutation_rate)
        offspring += mutations
        offspring = np.clip(offspring, bounds[:, 0], bounds[:, 1])
        population = np.vstack((selected, offspring))
    
    return best_points, all_populations

# Межі визначення функцій
bounds_dict = {
    'branin': np.array([[-5, 10], [0, 15]]),
    'easom': np.array([[-100, 100], [-100, 100]]),
    'goldstein_price': np.array([[-2, 2], [-2, 2]]),
    'six_hump_camel': np.array([[-3, 3], [-2, 2]])
}

functions = {
    'branin': branin,
    'easom': easom,
    'goldstein_price': goldstein_price,
    'six_hump_camel': six_hump_camel
}

In [30]:
best_points, all_populations = genetic_algorithm(branin, bounds_dict['branin'])

In [ ]:


def create_optimization_gif(func, bounds, best_points, all_populations, filename="optimization.gif"):
    x = np.linspace(bounds[0, 0], bounds[0, 1], 100)
    y = np.linspace(bounds[1, 0], bounds[1, 1], 100)
    X, Y = np.meshgrid(x, y)
    Z = func(X, Y)
    
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')
    
    def update(frame):
        ax.clear()
        ax.plot_surface(X, Y, Z, cmap='coolwarm', alpha=1)
        
        population = all_populations[frame]
        ax.scatter(population[:, 0], population[:, 1], [func(ind[0], ind[1]) for ind in population], color='red', s=5, label='Population')
        
        best_x, best_y = best_points[frame]
        best_z = func(best_x, best_y)
        ax.scatter(best_x, best_y, best_z, color='yellow', s=50, label=f'Best {best_z:.4f}')
        
        ax.set_xlabel("X")
        ax.set_ylabel("Y")
        ax.set_zlabel("Z")
        ax.set_title(f"Optimization Progress (Iteration {frame+1})")
        ax.legend()
    
    ani = animation.FuncAnimation(fig, update, frames=len(best_points), repeat=False)
    ani.save(filename, writer="pillow", fps=5)
    plt.close()


In [32]:
create_optimization_gif(branin, bounds_dict['branin'], best_points, all_populations, filename="branin_optimization.gif")